In [74]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTShadowClient

import random, time, datetime 
import pandas as pd
import json

In [75]:
SHADOW_CLIENT="A0186745N_2"
HOST_NAME="a2zdmpkl8brfji-ats.iot.us-west-2.amazonaws.com"
ROOT_CA="Amazon_Root_CA_2.pem"
PRIVATE_KEY="628b1f5d6c-private.pem.key"
CERT_FILE="628b1f5d6c-certificate.pem.crt"
SHADOW_HANDLER="A0186745N_2"

In [76]:
def myShadowUpdateCallback(payload,responseStatus,token):
    print()
    print('UPDATE:$aws/things/'+SHADOW_HANDLER+
          '/shadow/update/#')
    #print("payload="+payload)
    #print("responseStatus="+responseStatus)
    #print("token="+token)

In [77]:
def customShadowCallback_delete(payload, responseStatus, token):
    if responseStatus == "timeout":
        print("Delete request " + token + " time out!")
    if responseStatus == "accepted":
        print("Delete request with token " + token + " accepted!")
    if responseStatus == "rejected":
        print("Delete request with token " + token + " rejected!")

In [78]:
def customShadowCallback_Delta(payload, responseStatus, token):
    # payload is a JSON string which will be parsed by jason lib
    print(responseStatus)
    print(payload)
    payloadDict = json.loads(payload)
    print("++++++++ Get DELTA data +++++++++++")
    desiredStatus = str(payloadDict["reported"])
    print("desired status: " + desiredStatus)
    print("version: " + str(payloadDict["version"]))

In [79]:
def customShadowCallback_Get(payload, responseStatus, token):
    print("responseStatus: " + responseStatus)
    print("payload: " + payload)
    payloadDict = json.loads(payload)
    # {"state":{"desired":{"light":37},"delta":{"light":37}},"metadata":{"desired":{"light":{"timestamp":1533888405}}},"version":54

In [80]:
# set up shadow client
myShadowClient=AWSIoTMQTTShadowClient(SHADOW_CLIENT)
myShadowClient.configureEndpoint(HOST_NAME,8883)
myShadowClient.configureCredentials(ROOT_CA,PRIVATE_KEY,CERT_FILE)
# myMQTTClient=AWSIoTMQTTClient(SHADOW_CLIENT)
# myMQTTClient.configureEndpoint(HOST_NAME, 8883)
# myMQTTClient = myShadowClient.getMQTTConnection()
# myMQTTClient.configureOfflinePublishQueueing(-1)
myShadowClient.configureConnectDisconnectTimeout(10)
myShadowClient.configureMQTTOperationTimeout(10)
myShadowClient.connect()
print("connectted to AWS IoT")
myDeviceShadow=myShadowClient.createShadowHandlerWithName(SHADOW_HANDLER,True)

connectted to AWS IoT


In [81]:
#%% load library
import gc
#import resource
import rope
#import operator
import numpy as np
import pandas as pd

In [82]:
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set(style="white", color_codes=True)

#import xgboost as xgb
#from xgboost.sklearn import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV

import warnings
warnings.filterwarnings("ignore")

In [90]:
#%% Load data
print('Start loading data ...')

# Load and combine training data
train_2 = pd.read_csv('train_FD002.txt', delim_whitespace=True, header=None)
train=train_2
print(train)

Start loading data ...
        0    1        2       3      4       5       6        7        8   \
0        1    1  34.9983  0.8400  100.0  449.44  555.32  1358.61  1137.23   
1        1    2  41.9982  0.8408  100.0  445.00  549.90  1353.22  1125.78   
2        1    3  24.9988  0.6218   60.0  462.54  537.31  1256.76  1047.45   
3        1    4  42.0077  0.8416  100.0  445.00  549.51  1354.03  1126.38   
4        1    5  25.0005  0.6203   60.0  462.54  537.07  1257.71  1047.93   
5        1    6  25.0045  0.6205   60.0  462.54  537.02  1266.38  1048.72   
6        1    7  42.0043  0.8409  100.0  445.00  549.74  1347.45  1127.19   
7        1    8  20.0020  0.7002  100.0  491.19  607.44  1481.69  1252.36   
8        1    9  41.9995  0.8407  100.0  445.00  549.33  1348.23  1127.45   
9        1   10  42.0011  0.8400  100.0  445.00  549.33  1356.40  1127.11   
10       1   11  42.0029  0.8400  100.0  445.00  549.81  1352.72  1126.23   
11       1   12   0.0015  0.0010  100.0  518.67  642.

In [91]:
# Check dimension
print('Dimension of train_FD002: ', train.shape)

Dimension of train_FD002:  (53759, 26)


In [92]:
# Assign column headers: id, te, (time evolutions) os1, os2, os3, (operational settings) s1, s2, ..., s21 (sensors)
sensor_name = ['s'+ str(i) for i in range(1,22)]
train.columns = ['id', 'te', 'os1', 'os2', 'os3'] + sensor_name
train['id'] = train_2['id'].map(lambda s: 'FD002_'+str(s))
train['Maltricnum']='A0186745N'

In [93]:
import time
for i in range(5):
    test=train.loc[[i]] 
    now = datetime.datetime.utcnow()
    now='UTC'+str(now)
    test['timestamp']=now
    test.index=["reported"]
    test=test.to_json(orient='index')
    test='{"state":'+test+'}'
    jsonPayload=test    
    print(test)
#     myDeviceShadow.shadowUpdate(jsonPayload,myShadowUpdateCallback, 5)
#     time.sleep(10)

{"state":{"reported":{"id":"FD002_1","te":1,"os1":34.9983,"os2":0.84,"os3":100.0,"s1":449.44,"s2":555.32,"s3":1358.61,"s4":1137.23,"s5":5.48,"s6":8.0,"s7":194.64,"s8":2222.65,"s9":8341.91,"s10":1.02,"s11":42.02,"s12":183.06,"s13":2387.72,"s14":8048.56,"s15":9.3461,"s16":0.02,"s17":334,"s18":2223,"s19":100.0,"s20":14.73,"s21":8.8071,"Maltricnum":"A0186745N","timestamp":"UTC2019-09-19 15:51:02.677138"}}}
{"state":{"reported":{"id":"FD002_1","te":2,"os1":41.9982,"os2":0.8408,"os3":100.0,"s1":445.0,"s2":549.9,"s3":1353.22,"s4":1125.78,"s5":3.91,"s6":5.71,"s7":138.51,"s8":2211.57,"s9":8303.96,"s10":1.02,"s11":42.2,"s12":130.42,"s13":2387.66,"s14":8072.3,"s15":9.3774,"s16":0.02,"s17":330,"s18":2212,"s19":100.0,"s20":10.41,"s21":6.2665,"Maltricnum":"A0186745N","timestamp":"UTC2019-09-19 15:51:02.685149"}}}
{"state":{"reported":{"id":"FD002_1","te":3,"os1":24.9988,"os2":0.6218,"os3":60.0,"s1":462.54,"s2":537.31,"s3":1256.76,"s4":1047.45,"s5":7.05,"s6":9.02,"s7":175.71,"s8":1915.11,"s9":8001.42

In [88]:
print(test)

{"state":{"reported":{"id":"FD002_1","te":5,"os1":25.0005,"os2":0.6203,"os3":60.0,"s1":462.54,"s2":537.07,"s3":1257.71,"s4":1047.93,"s5":7.05,"s6":9.03,"s7":175.05,"s8":1915.1,"s9":7993.23,"s10":0.94,"s11":36.89,"s12":164.31,"s13":2028.0,"s14":7861.23,"s15":10.8963,"s16":0.02,"s17":309,"s18":1915,"s19":84.93,"s20":14.13,"s21":8.5286,"Maltricnum":"A0186745N","timestamp":"UTC2019-09-19 15:43:42.173334"}}}


In [54]:
myDeviceShadow.shadowDelete( customShadowCallback_delete, 5)

'44368c8b-07d7-4d99-a35f-dce41514d81c'

Delete request with token 44368c8b-07d7-4d99-a35f-dce41514d81c accepted!
